In [1]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

In [2]:
START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        eq('-'),
        eq('это')
    ),    
    rule(
        eq('являются')
    ),
    rule(
        eq('является')
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [3]:
def get_hyper(row):
    main_word = row['WORD']
    text = row['TEXT']
    
    HYPONYM = eq(utils.deaccent(main_word.lower()))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(HYPONYM, MID, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, END),
        rule(END, MID, END),
        rule(END, MID, MID, END),
    )
    parser = Parser(RULE) 
    text = utils.deaccent(text)
    # print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    hyps = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyps.append(k)
    print(main_word, hyps)
    return hyps

In [4]:
pwd

'C:\\Users\\User'

In [5]:
cd C:\\Users\\User\Desktop

C:\Users\User\Desktop


In [6]:
import pandas as pd

data = pd.read_csv('hyper2_2.csv', encoding='utf-8')
data.fillna('', inplace=True)
data.head(20)

,WORD,HYPERONYMS,TEXT
0,ЛЕКСИКОГРАФИЯ,Словообразование,"Лексикография — раздел языкознания, занимающий..."
1,ЛЕКСИКОГРАФИЯ,Языкознание,лексикография это собирание слов какого-нибуд...
2,ЛЕКСИКОГРАФИЯ,Собирание слов,ЛЕКСИКОГРАФИЯ это совокупность печатных трудо...
3,ЛЕКСИКОГРАФИЯ,Словосочетание,"лексикография — труды по лексикографии, а таже..."
4,ЛЕКСИКОГРАФИЯ,Раздел лингвистики,лексикография — это специальный раздел лингвис...
5,ЛЕМЕХ,Сошник,"Лемех — сошник у плуга, у сабана и косули, ра..."
6,ЛЕМЕХ,Пахарь,"Лемех это Пахарь, Стальной, служит для подрез..."
7,ЛЕМЕХ,Часть плуга,"Лемех — Часть плуга, подрезающая пласт земли ..."
8,ЛЕМЕХ,Наконечник плуга,Лемех — острый наконечник плуга.
9,ЛЕМЕХ,Деталь,Лемех — это острая деталь плуга служащая для п...


In [7]:
data['dec_hyper'] = data.apply(lambda row: get_hyper(row), axis=1)

ЛЕКСИКОГРАФИЯ [['лексикография', '—', 'раздел', 'языкознания']]
ЛЕКСИКОГРАФИЯ [['лексикография', 'это', 'собирание', 'слов']]
ЛЕКСИКОГРАФИЯ [['лексикография', 'это', 'совокупность', 'печатных', 'трудов']]
ЛЕКСИКОГРАФИЯ [['лексикография', '—', 'труды']]
ЛЕКСИКОГРАФИЯ [['лексикография', '—', 'это', 'специальныи', 'раздел', 'лингвистики']]
ЛЕМЕХ [['лемех', '—', 'сошник'], ['лемех', 'треугольныи']]
ЛЕМЕХ [['лемех', 'это', 'пахарь']]
ЛЕМЕХ [['лемех', '—', 'часть', 'плуга']]
ЛЕМЕХ [['лемех', '—', 'острыи', 'наконечник', 'плуга']]
ЛЕМЕХ [['лемех', '—', 'это', 'острая', 'деталь']]
ЛЕМУР [['лемур', '—', 'разряд', 'обезьян']]
ЛЕМУР [['лемур', '—', 'полуобезьяна', 'с', 'длинным', 'хвостом', 'и']]
ЛЕМУР [['лемур', '—', 'небольшои', 'зверек', 'тропических', 'лесов']]
ЛЕМУР [['лемур', '—', 'представитель', 'инфраотряда', 'приматов', 'лемурообразные']]
ЛЕМУР [['лемур', '—', 'представитель', 'приматов']]
ЛЕНИНЕЦ [['ленинец', 'это', 'человек']]
ЛЕНИНЕЦ [['лениец', '—', 'последователь', 'ленинизма']]
ЛЕ